In [68]:
import shutil
import urllib.request as request
from contextlib import closing

# first we download the Sift1M dataset
with closing(request.urlopen('ftp://ftp.irisa.fr/local/texmex/corpus/sift.tar.gz')) as r:
    with open('sift.tar.gz', 'wb') as f:
        shutil.copyfileobj(r, f)

In [69]:
import tarfile

# the download leaves us with a tar.gz file, we unzip it
tar = tarfile.open('sift.tar.gz', "r:gz")
tar.extractall()

In [70]:
import numpy as np

def read_fvecs(fp):
  a = np.fromfile(fp, dtype='int32')
  d = a[0]
  return a.reshape(-1, d + 1)[:, 1:].copy().view('float32')

# data we will search through
xb = read_fvecs('./sift/sift_base.fvecs')  # 1M samples
# also get some query vectors to search with
xq = read_fvecs('./sift/sift_query.fvecs')
# take just one query (there are many in sift_learn.fvecs)
xq = xq[0].reshape(1, xq.shape[1])

In [71]:
print(xq.shape)
print(xb.shape)

(1, 128)
(1000000, 128)


In [72]:
# number of hyperplanes
nbits = 4
d = 2  # vector dimensionality

plane_norms = np.random.rand(nbits, d)  - .5
plane_norms

array([[-0.39177222,  0.12706225],
       [ 0.2943267 , -0.46770264],
       [-0.13975001,  0.21163821],
       [ 0.28047369, -0.47994448]])

In [73]:
a = np.asarray([1, 2])
b = np.asarray([2, 1])
c = np.asarray([3, 1])

In [74]:
a_dot = np.dot(a, plane_norms.T)
b_dot = np.dot(b, plane_norms.T)
c_dot = np.dot(c, plane_norms.T)

In [75]:
a_dot = a_dot > 0
b_dot = b_dot > 0
c_dot = c_dot > 0
a_dot

array([False, False,  True, False])

In [76]:
a_dot = a_dot.astype(int)
b_dot = b_dot.astype(int)
c_dot = c_dot.astype(int)

print(a_dot)
print(b_dot)
print(c_dot)

[0 0 1 0]
[0 1 0 1]
[0 1 0 1]


In [77]:
vectors = [a_dot, b_dot, c_dot]
buckets = {}

for i in range(len(vectors)):
  hash_str = ''.join(vectors[i].astype(str))
  if hash_str not in buckets.keys():
    buckets[hash_str] = []
  buckets[hash_str].append(i)

print(buckets)

{'0010': [0], '0101': [1, 2]}


In [78]:
!pip install faiss-cpu

In [79]:
import faiss

In [80]:
xb

array([[  0.,  16.,  35., ...,  25.,  23.,   1.],
       [ 14.,  35.,  19., ...,  11.,  21.,  33.],
       [  0.,   1.,   5., ...,   4.,  23.,  10.],
       ...,
       [ 30.,  12.,  12., ...,  50.,  10.,   0.],
       [  0.,   5.,  12., ...,   1.,   2.,  13.],
       [114.,  31.,   0., ...,  25.,  16.,   0.]], dtype=float32)

In [81]:
d = xb.shape[1]
d

128

In [82]:
nbits = 4

In [83]:
# how many buckets are gives us
2**nbits

16

In [84]:
index = faiss.IndexLSH(d, nbits)  # d = xb.shape[1]

In [85]:
index.is_trained

True

In [86]:
index.add(xb)

In [87]:
k = 10
D, I = index.search(xq, k)

In [88]:
I

array([[ 0,  2,  6, 25, 26, 43, 47, 70, 73, 74]])

In [89]:
D

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [90]:
from sklearn.metrics.pairwise import cosine_similarity

cos = cosine_similarity(xb[I[0]], xq)
cos.mean()

0.44347006

In [91]:
from sklearn.metrics.pairwise import cosine_similarity

cos = cosine_similarity(xb, xq)
cos.mean()

0.4223302

In [92]:
for nbits in [2, 4, 8, 16, 24, 32]:
    buckets = 1 << nbits
    print(f"nbits == {nbits}")
    print(f"{xb.shape[0]} / {buckets} = {xb.shape[0]/buckets}")

nbits == 2
1000000 / 4 = 250000.0
nbits == 4
1000000 / 16 = 62500.0
nbits == 8
1000000 / 256 = 3906.25
nbits == 16
1000000 / 65536 = 15.2587890625
nbits == 24
1000000 / 16777216 = 0.059604644775390625
nbits == 32
1000000 / 4294967296 = 0.00023283064365386963


In [93]:
xq0 = xq[0].reshape(1, d)
k = 100

for nbits in [2, 4, 8, 16, 24, 32]:
    index = faiss.IndexLSH(d, nbits)
    index.add(xb)
    D, I = index.search(xq0, k=k)
    cos = cosine_similarity(xb[I[0]], xq0)
    print(np.mean(cos))

0.54244477
0.56082696
0.6372648
0.6676912
0.7132521
0.7051426
